In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

#Defining various data and distacne matrix


I=['1','2','3','4','5','6','7','8','9'] # Demand ID
J=['A','B','C','D','E','F']   #EMS ID
A=[300,5,6,2,5,1,3,5,6]  # the amount of demand at each locaton
R=[0,1,1,1,0,0,0,0,0]  # whether the demand location is located in rural areas： 1-Yes, 0-No

M= 320 # total urban demand
N= 13 # total rural demand
tot_pop=M+N # total demand in the study area
q=1   #total number of EMS stations to be built 


#########################################################################################
#O-D distance matrix 
#J         A B  C  D  E  F
D=       [[0,10,12,10,19,10],#1 urban
          [12,10,9,10,10,26],#2 rural 
          [12,13,10,8,17,10],#3 rural 
          [10,10,18,10,13,7],#4 rural 
          [19,17,7,13,10,19],#5 urban 
          [17,26,15,14,9,10],#6 urban
          [19,10,16,5,12,15],#7 urban
          [10,13,16,10,7,14],#8 urban
          [12,15,6,18,10,13]]#9 urban 

# coverage=9 if location is covered -1, not covered -0

C=       [[1,0,0,0,0,0],#1 
          [0,0,1,0,0,0],#2 -
          [0,0,0,1,0,0],#3 -
          [0,0,0,0,0,1],#4 
          [0,0,1,0,0,0],#5 -
          [0,0,0,0,1,0],#6 
          [0,0,0,1,0,0],#7-
          [0,0,0,0,1,0],#8
          [0,0,1,0,0,0]]#9-

# if location is covered -1, not covered 0, 
UC=      [[0,1,1,1,1,1],#1
          [1,1,0,1,1,1],#2 
          [1,1,1,0,1,1],#3
          [1,1,1,1,1,0],#4
          [1,1,0,1,1,1],#5
          [1,1,1,1,0,1],#6
          [1,1,1,0,1,1],#7
          [1,1,1,1,0,1],#8
          [1,1,0,1,1,1]]#9

print("finish")



finish


In [4]:
#  Model 1 - maximize the amount of covered demand. 
#we aim to build two EMS station, that is q=2

q=2# upgrade the number of stations

import gurobipy as gp
from gurobipy import GRB
import numpy as np
m=gp.Model("Urban_rural inequality model_ p median ")

#define decision variables  
X = m.addVars(range(1,len(J)+1),vtype=GRB.BINARY,name="Xj") # covergae decision
Y = m.addVars(range(1,len(I)+1),range(1,len(J)+1),vtype=GRB.INTEGER,lb=0,name="Yij") #Allocation variables
Wr= m.addVar(vtype= GRB.CONTINUOUS,name="Wr")# rural covered ratio
Wu= m.addVar(vtype=GRB.CONTINUOUS,name="Wu")# urban covered ratio

# Define constraints:
#SumXj +Sum(uc)Yij >=1
m.addConstrs(gp.quicksum(X[j]*C[i-1][j-1] for j in range(1,len(J)+1))+
             gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1))>=1 for i in range(1, len(I)+1))

#Sum Xj=q
m.addConstr(X.sum()==q)

# X[j]-Y[i,j]>=0 for i in I, j in UC
m.addConstrs(X[j]*UC[i-1][j-1]-Y[i,j]*UC[i-1][j-1]>=0 for i in range(1,len(I)+1) for j in range(1,len(J)+1))


#Wu
m.addConstr(gp.quicksum(A[i-1]*(1-R[i-1])*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1)))
                        for i in range(1,len(I)+1))/ M -Wu==0)

#Wr
m.addConstr(gp.quicksum(A[i-1]*R[i-1]*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1)))
                        for i in range(1,len(I)+1))/ N -Wr==0)


#Define objective
obj1=gp.quicksum(A[i-1]*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1,len(J)+1))) for i in range(1,len(I)+1))
m.setObjective(obj1, GRB.MAXIMIZE)

#Solve the model 
m.optimize()

for v in m.getVars():
       if v.X > 0.01:
        print("%s %g" % (v.Varname, v.X))
#print("obj2",obj2.getValue())    
print("obj1:", obj1.getValue())
print("finish")



Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 66 rows, 62 columns and 197 nonzeros
Model fingerprint: 0x431eac0a
Variable types: 2 continuous, 60 integer (6 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+00]
  Objective range  [1e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 9 rows and 11 columns
Presolve time: 0.00s
Presolved: 57 rows, 51 columns, 195 nonzeros
Variable types: 0 continuous, 51 integer (51 binary)

Root relaxation: objective 3.160000e+02, 19 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     316.0000000  316.00000  0.00%     -    0s

Explored 0 nodes (19 simplex iterations) in 0.01 seconds
Thread co

In [7]:
#Model 2. Minimize the total weighted distace for uncovered rural population 
#we aim to locate one EMS station in this scenario (that is q=1)
q=1# upgraded the q

import gurobipy as gp
from gurobipy import GRB
import numpy as np
m=gp.Model("Urban_rural inequality model_ p median ")

# define decision variables 
X = m.addVars(range(1,len(J)+1),vtype=GRB.BINARY,name="Xj") # covergae decision
Y = m.addVars(range(1,len(I)+1),range(1,len(J)+1),vtype=GRB.INTEGER,lb=0,name="Yij") #Allocation variables
Wr= m.addVar(vtype= GRB.CONTINUOUS,name="Wr")
Wu= m.addVar(vtype=GRB.CONTINUOUS,name="Wu")

# Define constraints:
#SumXj +Sum(uc)Yij >=1
m.addConstrs(gp.quicksum(X[j]*C[i-1][j-1] for j in range(1,len(J)+1))+
             gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1))>=1 for i in range(1, len(I)+1))

#Sum Xj=q
m.addConstr(X.sum()==q)


# X[j]-Y[i,j]>=0 for i in I, j in UC
m.addConstrs(X[j]*UC[i-1][j-1]-Y[i,j]*UC[i-1][j-1]>=0 for i in range(1,len(I)+1) for j in range(1,len(J)+1))


#Wu
m.addConstr(gp.quicksum(A[i-1]*(1-R[i-1])*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1)))
                        for i in range(1,len(I)+1))/ M -Wu==0)

#Wr
m.addConstr(gp.quicksum(A[i-1]*R[i-1]*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1)))
                        for i in range(1,len(I)+1))/ N -Wr==0)

#Define the objective
obj2 = gp.quicksum(gp.quicksum(A[i-1]*R[i-1]*D[i-1][j-1]*UC[i-1][j-1]*Y[i,j]for j in range(1, len(J)+1))
              for i in range(1,len(I)+1))

obj1=gp.quicksum(A[i-1]*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1,len(J)+1))) for i in range(1,len(I)+1))


m.modelSense = GRB.MINIMIZE
m.setObjective(obj2, GRB.MINIMIZE)

#Solve the model 
m.optimize()


for v in m.getVars():
       if v.X > 0.01:
        print("%s %g" % (v.Varname, v.X))
print("obj2",obj2.getValue()) 
print("obj1",obj1.getValue())

print("finish")



Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 66 rows, 62 columns and 197 nonzeros
Model fingerprint: 0x1f308451
Variable types: 2 continuous, 60 integer (6 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+00]
  Objective range  [2e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 148.0000000
Presolve removed 66 rows and 62 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 70 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.000000000000e+01, best bound 7.000000000000e+01, gap 0.0000%
Xj[4] 1
Yij[1,4] 1
Yij[2,4] 1
Yij[4,4] 1
Yij[5,4] 1
Yij[6,4] 1
Yij[8,4] 1
Yij[9,4] 1
Wr 0.461538
obj2 70.0
obj1 9.0
finish


In [6]:
#Model 3. Minimize the total weight distace for uncovered rural population, while the amuont of covered people should be not less than 10
# we aim to locate one EMS station in thise scenario (that is q=1)
q=1

import gurobipy as gp
from gurobipy import GRB
import numpy as np
m=gp.Model("Model 3 ")

X = m.addVars(range(1,len(J)+1),vtype=GRB.BINARY,name="Xj") # covergae decision
Y = m.addVars(range(1,len(I)+1),range(1,len(J)+1),vtype=GRB.INTEGER,lb=0,name="Yij") #Allocation variables
Wr= m.addVar(vtype= GRB.CONTINUOUS,name="Wr")
Wu= m.addVar(vtype=GRB.CONTINUOUS,name="Wu")


# Constraints:
#SumXj +Sum(uc)Yij >=1
m.addConstrs(gp.quicksum(X[j]*C[i-1][j-1] for j in range(1,len(J)+1))+
             gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1))>=1 for i in range(1, len(I)+1))

#Sum Xj=q
m.addConstr(X.sum()==q)


# X[j]-Y[i,j]>=0 for i in I, j in UC
m.addConstrs(X[j]*UC[i-1][j-1]-Y[i,j]*UC[i-1][j-1]>=0 for i in range(1,len(I)+1) for j in range(1,len(J)+1))


#Wu
m.addConstr(gp.quicksum(A[i-1]*(1-R[i-1])*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1)))
                        for i in range(1,len(I)+1))/ M -Wu==0)

#Wr
m.addConstr(gp.quicksum(A[i-1]*R[i-1]*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1)))
                        for i in range(1,len(I)+1))/ N -Wr==0)

#Define objectives

obj2 = gp.quicksum(gp.quicksum(A[i-1]*R[i-1]*D[i-1][j-1]*UC[i-1][j-1]*Y[i,j]for j in range(1, len(J)+1))
              for i in range(1,len(I)+1))

obj1=gp.quicksum(A[i-1]*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1,len(J)+1))) for i in range(1,len(I)+1))

# e-constraints method
m.addConstr(obj1>=10)
#Soleve the model 
m.modelSense = GRB.MINIMIZE
m.setObjective(obj2, GRB.MINIMIZE)
m.optimize()

for v in m.getVars():
       if v.X > 0.001:
        print("%s %g" % (v.Varname, v.X))
print("obj2",obj2.getValue()) 
print("obj1",obj1.getValue())

print("finish")

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 67 rows, 62 columns and 242 nonzeros
Model fingerprint: 0xe8c6c504
Variable types: 2 continuous, 60 integer (6 binary)
Coefficient statistics:
  Matrix range     [3e-03, 3e+02]
  Objective range  [2e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 67 rows and 62 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 96 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.600000000000e+01, best bound 9.600000000000e+01, gap 0.0000%
Xj[3] 1
Yij[1,3] 1
Yij[3,3] 1
Yij[4,3] 1
Yij[6,3] 1
Yij[7,3] 1
Yij[8,3] 1
Wr 0.384615
Wu 0.034375
obj2 96.0
obj1 16.0
finish


In [5]:
#Model 4.  maximize the total amount of covered demand, while Wr should be higher than 40%. 
#we aim to build two EMS stations (that is q=2) 

import gurobipy as gp 
from gurobipy import GRB
import numpy as np
m=gp.Model("Model 4 ")

#Define decsion variables
X = m.addVars(range(1,len(J)+1),vtype=GRB.BINARY,name="Xj") # covergae decision
Y = m.addVars(range(1,len(I)+1),range(1,len(J)+1),vtype=GRB.INTEGER,lb=0,name="Yij") #Allocation variables
Wr= m.addVar(vtype= GRB.CONTINUOUS,name="Wr")
Wu= m.addVar(vtype=GRB.CONTINUOUS,name="Wu")

# Constraints:
#SumXj +Sum(uc)Yij >=1
m.addConstrs(gp.quicksum(X[j]*C[i-1][j-1] for j in range(1,len(J)+1))+
             gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1))>=1 for i in range(1, len(I)+1))

#Sum Xj=q
m.addConstr(X.sum()==q)

# X[j]-Y[i,j]>=0 for i in I, j in UC
m.addConstrs(X[j]*UC[i-1][j-1]-Y[i,j]*UC[i-1][j-1]>=0 for i in range(1,len(I)+1) for j in range(1,len(J)+1))


#Wu
m.addConstr(gp.quicksum(A[i-1]*(1-R[i-1])*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1)))
                        for i in range(1,len(I)+1))/ M -Wu==0)

#Wr
m.addConstr(gp.quicksum(A[i-1]*R[i-1]*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1)))
                        for i in range(1,len(I)+1))/ N -Wr==0)

#Define objectives 
obj1=gp.quicksum(A[i-1]*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1,len(J)+1))) for i in range(1,len(I)+1))

#e-constraints method

m.addConstr(Wr>=0.4)
#Solve the model 
m.setObjective(obj1, GRB.MAXIMIZE)
m.optimize()
for v in m.getVars():
       if v.X > 0.01:
        print("%s %g" % (v.Varname, v.X)) 
print("obj1:", obj1.getValue())
print("finish")



Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 67 rows, 62 columns and 198 nonzeros
Model fingerprint: 0x825434d0
Variable types: 2 continuous, 60 integer (6 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+00]
  Objective range  [1e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-01, 2e+00]
Presolve removed 15 rows and 14 columns
Presolve time: 0.00s
Presolved: 52 rows, 48 columns, 181 nonzeros
Variable types: 0 continuous, 48 integer (48 binary)
Found heuristic solution: objective 309.0000000

Root relaxation: cutoff, 22 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0       309.00000  309.00000  0.00%     -    0s

Explored 0 nodes (22 simplex iterations) in 0.00 seconds
Thread count was 8 (of